# EVM logs decode

In [ ]:
#!pip install web3

In [1]:
import json
import pandas as pd
from web3 import Web3
from web3.middleware import geth_poa_middleware
from web3.logs import STRICT, IGNORE, DISCARD, WARN
from web3.datastructures import AttributeDict
from web3._utils.events import get_event_data
from web3.exceptions import (InvalidEventABI, LogTopicError, MismatchedABI)
from hexbytes import HexBytes
from eth_abi.codec import ABICodec
from eth_abi.registry import registry as default_registry
from eth_utils import (encode_hex, event_abi_to_log_topic)


In [2]:
default_codec = ABICodec(default_registry)

def get_event_abi(abi, abi_name, abi_type = 'event'):
    l = [x for x in abi if x['type'] == abi_type and x['name']==abi_name]
    return l[0]
def get_events(abi_dict, abi_type = 'event'):
    events = []
    for contract_type in abi_dict.keys():
        for event in [x for x in abi_dict[contract_type] if x['type'] == 'event']:
            events.append({'contract_type': contract_type,
                           'event'        : event['name'],
                           'topic'        : encode_hex(event_abi_to_log_topic(event)),
                           'abi'          : event})
    
    df= pd.DataFrame(events)
    return df

In [3]:
w3 = Web3(Web3.HTTPProvider('https://polygon-rpc.com/', request_kwargs={'timeout': 10}))

# inject the poa compatibility middleware to the innermost layer
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

ocr_abi_str='''
[{"inputs":[{"internalType":"uint32","name":"_maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"_reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"_microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerTransmission","type":"uint32"},{"internalType":"address","name":"_link","type":"address"},{"internalType":"address","name":"_validator","type":"address"},{"internalType":"int192","name":"_minAnswer","type":"int192"},{"internalType":"int192","name":"_maxAnswer","type":"int192"},{"internalType":"contract AccessControllerInterface","name":"_billingAccessController","type":"address"},{"internalType":"contract AccessControllerInterface","name":"_requesterAccessController","type":"address"},{"internalType":"uint8","name":"_decimals","type":"uint8"},{"internalType":"string","name":"description","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"user","type":"address"}],"name":"AddedAccess","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"int256","name":"current","type":"int256"},{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"updatedAt","type":"uint256"}],"name":"AnswerUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"contract AccessControllerInterface","name":"old","type":"address"},{"indexed":false,"internalType":"contract AccessControllerInterface","name":"current","type":"address"}],"name":"BillingAccessControllerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint32","name":"maximumGasPrice","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"reasonableGasPrice","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"microLinkPerEth","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"linkGweiPerObservation","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"linkGweiPerTransmission","type":"uint32"}],"name":"BillingSet","type":"event"},{"anonymous":false,"inputs":[],"name":"CheckAccessDisabled","type":"event"},{"anonymous":false,"inputs":[],"name":"CheckAccessEnabled","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint32","name":"previousConfigBlockNumber","type":"uint32"},{"indexed":false,"internalType":"uint64","name":"configCount","type":"uint64"},{"indexed":false,"internalType":"address[]","name":"signers","type":"address[]"},{"indexed":false,"internalType":"address[]","name":"transmitters","type":"address[]"},{"indexed":false,"internalType":"uint8","name":"threshold","type":"uint8"},{"indexed":false,"internalType":"uint64","name":"encodedConfigVersion","type":"uint64"},{"indexed":false,"internalType":"bytes","name":"encoded","type":"bytes"}],"name":"ConfigSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":true,"internalType":"address","name":"startedBy","type":"address"},{"indexed":false,"internalType":"uint256","name":"startedAt","type":"uint256"}],"name":"NewRound","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint32","name":"aggregatorRoundId","type":"uint32"},{"indexed":false,"internalType":"int192","name":"answer","type":"int192"},{"indexed":false,"internalType":"address","name":"transmitter","type":"address"},{"indexed":false,"internalType":"int192[]","name":"observations","type":"int192[]"},{"indexed":false,"internalType":"bytes","name":"observers","type":"bytes"},{"indexed":false,"internalType":"bytes32","name":"rawReportContext","type":"bytes32"}],"name":"NewTransmission","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"transmitter","type":"address"},{"indexed":false,"internalType":"address","name":"payee","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"OraclePaid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"transmitter","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"},{"indexed":true,"internalType":"address","name":"proposed","type":"address"}],"name":"PayeeshipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"transmitter","type":"address"},{"indexed":true,"internalType":"address","name":"previous","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"}],"name":"PayeeshipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"user","type":"address"}],"name":"RemovedAccess","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"contract AccessControllerInterface","name":"old","type":"address"},{"indexed":false,"internalType":"contract AccessControllerInterface","name":"current","type":"address"}],"name":"RequesterAccessControllerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"requester","type":"address"},{"indexed":false,"internalType":"bytes16","name":"configDigest","type":"bytes16"},{"indexed":false,"internalType":"uint32","name":"epoch","type":"uint32"},{"indexed":false,"internalType":"uint8","name":"round","type":"uint8"}],"name":"RoundRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previous","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"}],"name":"ValidatorUpdated","type":"event"},{"inputs":[],"name":"LINK","outputs":[{"internalType":"contract LinkTokenInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"acceptOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"acceptPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"}],"name":"addAccess","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"billingAccessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"checkEnabled","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"disableAccessCheck","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"enableAccessCheck","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBilling","outputs":[{"internalType":"uint32","name":"maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"linkGweiPerTransmission","type":"uint32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"},{"internalType":"bytes","name":"_calldata","type":"bytes"}],"name":"hasAccess","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestConfigDetails","outputs":[{"internalType":"uint32","name":"configCount","type":"uint32"},{"internalType":"uint32","name":"blockNumber","type":"uint32"},{"internalType":"bytes16","name":"configDigest","type":"bytes16"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRound","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTransmissionDetails","outputs":[{"internalType":"bytes16","name":"configDigest","type":"bytes16"},{"internalType":"uint32","name":"epoch","type":"uint32"},{"internalType":"uint8","name":"round","type":"uint8"},{"internalType":"int192","name":"latestAnswer","type":"int192"},{"internalType":"uint64","name":"latestTimestamp","type":"uint64"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"linkAvailableForPayment","outputs":[{"internalType":"int256","name":"availableBalance","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"maxAnswer","outputs":[{"internalType":"int192","name":"","type":"int192"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"minAnswer","outputs":[{"internalType":"int192","name":"","type":"int192"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_signerOrTransmitter","type":"address"}],"name":"oracleObservationCount","outputs":[{"internalType":"uint16","name":"","type":"uint16"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"owedPayment","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address payable","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"}],"name":"removeAccess","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"requestNewRound","outputs":[{"internalType":"uint80","name":"","type":"uint80"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"requesterAccessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint32","name":"_maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"_reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"_microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerTransmission","type":"uint32"}],"name":"setBilling","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract AccessControllerInterface","name":"_billingAccessController","type":"address"}],"name":"setBillingAccessController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"_signers","type":"address[]"},{"internalType":"address[]","name":"_transmitters","type":"address[]"},{"internalType":"uint8","name":"_threshold","type":"uint8"},{"internalType":"uint64","name":"_encodedConfigVersion","type":"uint64"},{"internalType":"bytes","name":"_encoded","type":"bytes"}],"name":"setConfig","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"_transmitters","type":"address[]"},{"internalType":"address[]","name":"_payees","type":"address[]"}],"name":"setPayees","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract AccessControllerInterface","name":"_requesterAccessController","type":"address"}],"name":"setRequesterAccessController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_newValidator","type":"address"}],"name":"setValidator","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_to","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"},{"internalType":"address","name":"_proposed","type":"address"}],"name":"transferPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes","name":"_report","type":"bytes"},{"internalType":"bytes32[]","name":"_rs","type":"bytes32[]"},{"internalType":"bytes32[]","name":"_ss","type":"bytes32[]"},{"internalType":"bytes32","name":"_rawVs","type":"bytes32"}],"name":"transmit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"transmitters","outputs":[{"internalType":"address[]","name":"","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"validator","outputs":[{"internalType":"contract AggregatorValidatorInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_recipient","type":"address"},{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"withdrawFunds","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"withdrawPayment","outputs":[],"stateMutability":"nonpayable","type":"function"}]
'''
keepers_abi_str='''
[{"inputs":[{"internalType":"address","name":"link","type":"address"},{"internalType":"address","name":"linkEthFeed","type":"address"},{"internalType":"address","name":"fastGasFeed","type":"address"},{"internalType":"uint32","name":"paymentPremiumPPB","type":"uint32"},{"internalType":"uint24","name":"blockCountPerTurn","type":"uint24"},{"internalType":"uint32","name":"checkGasLimit","type":"uint32"},{"internalType":"uint24","name":"stalenessSeconds","type":"uint24"},{"internalType":"uint16","name":"gasCeilingMultiplier","type":"uint16"},{"internalType":"uint256","name":"fallbackGasPrice","type":"uint256"},{"internalType":"uint256","name":"fallbackLinkPrice","type":"uint256"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint32","name":"paymentPremiumPPB","type":"uint32"},{"indexed":false,"internalType":"uint24","name":"blockCountPerTurn","type":"uint24"},{"indexed":false,"internalType":"uint32","name":"checkGasLimit","type":"uint32"},{"indexed":false,"internalType":"uint24","name":"stalenessSeconds","type":"uint24"},{"indexed":false,"internalType":"uint16","name":"gasCeilingMultiplier","type":"uint16"},{"indexed":false,"internalType":"uint256","name":"fallbackGasPrice","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"fallbackLinkPrice","type":"uint256"}],"name":"ConfigSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"},{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"uint96","name":"amount","type":"uint96"}],"name":"FundsAdded","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":false,"internalType":"address","name":"to","type":"address"}],"name":"FundsWithdrawn","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address[]","name":"keepers","type":"address[]"},{"indexed":false,"internalType":"address[]","name":"payees","type":"address[]"}],"name":"KeepersUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Paused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"keeper","type":"address"},{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"PayeeshipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"keeper","type":"address"},{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"PayeeshipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"keeper","type":"address"},{"indexed":true,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"address","name":"payee","type":"address"}],"name":"PaymentWithdrawn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"RegistrarChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"account","type":"address"}],"name":"Unpaused","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"},{"indexed":true,"internalType":"uint64","name":"atBlockHeight","type":"uint64"}],"name":"UpkeepCanceled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"},{"indexed":true,"internalType":"bool","name":"success","type":"bool"},{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"uint96","name":"payment","type":"uint96"},{"indexed":false,"internalType":"bytes","name":"performData","type":"bytes"}],"name":"UpkeepPerformed","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"},{"indexed":false,"internalType":"uint32","name":"executeGas","type":"uint32"},{"indexed":false,"internalType":"address","name":"admin","type":"address"}],"name":"UpkeepRegistered","type":"event"},{"inputs":[],"name":"FAST_GAS_FEED","outputs":[{"internalType":"contract AggregatorV3Interface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"LINK","outputs":[{"internalType":"contract LinkTokenInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"LINK_ETH_FEED","outputs":[{"internalType":"contract AggregatorV3Interface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"acceptOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"keeper","type":"address"}],"name":"acceptPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"uint96","name":"amount","type":"uint96"}],"name":"addFunds","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"id","type":"uint256"}],"name":"cancelUpkeep","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"address","name":"from","type":"address"}],"name":"checkUpkeep","outputs":[{"internalType":"bytes","name":"performData","type":"bytes"},{"internalType":"uint256","name":"maxLinkPayment","type":"uint256"},{"internalType":"uint256","name":"gasLimit","type":"uint256"},{"internalType":"uint256","name":"adjustedGasWei","type":"uint256"},{"internalType":"uint256","name":"linkEth","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getCanceledUpkeepList","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getConfig","outputs":[{"internalType":"uint32","name":"paymentPremiumPPB","type":"uint32"},{"internalType":"uint24","name":"blockCountPerTurn","type":"uint24"},{"internalType":"uint32","name":"checkGasLimit","type":"uint32"},{"internalType":"uint24","name":"stalenessSeconds","type":"uint24"},{"internalType":"uint16","name":"gasCeilingMultiplier","type":"uint16"},{"internalType":"uint256","name":"fallbackGasPrice","type":"uint256"},{"internalType":"uint256","name":"fallbackLinkPrice","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"query","type":"address"}],"name":"getKeeperInfo","outputs":[{"internalType":"address","name":"payee","type":"address"},{"internalType":"bool","name":"active","type":"bool"},{"internalType":"uint96","name":"balance","type":"uint96"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getKeeperList","outputs":[{"internalType":"address[]","name":"","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"gasLimit","type":"uint256"}],"name":"getMaxPaymentForGas","outputs":[{"internalType":"uint96","name":"maxPayment","type":"uint96"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"id","type":"uint256"}],"name":"getMinBalanceForUpkeep","outputs":[{"internalType":"uint96","name":"minBalance","type":"uint96"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getRegistrar","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"id","type":"uint256"}],"name":"getUpkeep","outputs":[{"internalType":"address","name":"target","type":"address"},{"internalType":"uint32","name":"executeGas","type":"uint32"},{"internalType":"bytes","name":"checkData","type":"bytes"},{"internalType":"uint96","name":"balance","type":"uint96"},{"internalType":"address","name":"lastKeeper","type":"address"},{"internalType":"address","name":"admin","type":"address"},{"internalType":"uint64","name":"maxValidBlocknumber","type":"uint64"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getUpkeepCount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"sender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"onTokenTransfer","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"paused","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"bytes","name":"performData","type":"bytes"}],"name":"performUpkeep","outputs":[{"internalType":"bool","name":"success","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"recoverFunds","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"target","type":"address"},{"internalType":"uint32","name":"gasLimit","type":"uint32"},{"internalType":"address","name":"admin","type":"address"},{"internalType":"bytes","name":"checkData","type":"bytes"}],"name":"registerUpkeep","outputs":[{"internalType":"uint256","name":"id","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint32","name":"paymentPremiumPPB","type":"uint32"},{"internalType":"uint24","name":"blockCountPerTurn","type":"uint24"},{"internalType":"uint32","name":"checkGasLimit","type":"uint32"},{"internalType":"uint24","name":"stalenessSeconds","type":"uint24"},{"internalType":"uint16","name":"gasCeilingMultiplier","type":"uint16"},{"internalType":"uint256","name":"fallbackGasPrice","type":"uint256"},{"internalType":"uint256","name":"fallbackLinkPrice","type":"uint256"}],"name":"setConfig","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"keepers","type":"address[]"},{"internalType":"address[]","name":"payees","type":"address[]"}],"name":"setKeepers","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"registrar","type":"address"}],"name":"setRegistrar","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_to","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"keeper","type":"address"},{"internalType":"address","name":"proposed","type":"address"}],"name":"transferPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"unpause","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"address","name":"to","type":"address"}],"name":"withdrawFunds","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"}],"name":"withdrawPayment","outputs":[],"stateMutability":"nonpayable","type":"function"}]
'''
ocr_abi = json.loads(ocr_abi_str)
keepers_abi = json.loads(keepers_abi_str)

In [4]:
event_topics = get_events({'OCR':ocr_abi, 'Keepers': keepers_abi})
event_topics

,contract_type,event,topic,abi
0,OCR,AddedAccess,0x87286ad1f399c8e82bf0c4ef4fcdc570ea2e1e92176e...,"{'anonymous': False, 'inputs': [{'indexed': Fa..."
1,OCR,AnswerUpdated,0x0559884fd3a460db3073b7fc896cc77986f16e378210...,"{'anonymous': False, 'inputs': [{'indexed': Tr..."
2,OCR,BillingAccessControllerSet,0x793cb73064f3c8cde7e187ae515511e6e56d1ee89bf0...,"{'anonymous': False, 'inputs': [{'indexed': Fa..."
3,OCR,BillingSet,0xd0d9486a2c673e2a4b57fc82e4c8a556b3e2b82dd5db...,"{'anonymous': False, 'inputs': [{'indexed': Fa..."
4,OCR,CheckAccessDisabled,0x3be8a977a014527b50ae38adda80b56911c267328965...,"{'anonymous': False, 'inputs': [], 'name': 'Ch..."
5,OCR,CheckAccessEnabled,0xaebf329500988c6488a0074e5a0a9ff304561fc5c6fc...,"{'anonymous': False, 'inputs': [], 'name': 'Ch..."
6,OCR,ConfigSet,0x25d719d88a4512dd76c7442b910a83360845505894eb...,"{'anonymous': False, 'inputs': [{'indexed': Fa..."
7,OCR,NewRound,0x0109fc6f55cf40689f02fbaad7af7fe7bbac8a3d2186...,"{'anonymous': False, 'inputs': [{'indexed': Tr..."
8,OCR,NewTransmission,0xf6a97944f31ea060dfde0566e4167c1a1082551e64b6...,"{'anonymous': False, 'inputs': [{'indexed': Tr..."
9,OCR,OraclePaid,0xe8ec50e5150ae28ae37e493ff389ffab7ffaec2dc4dc...,"{'anonymous': False, 'inputs': [{'indexed': Fa..."


## <font color='blue'> 1. If we have a transaction hash </font>

In [5]:
txhash = '0x6bf6b3a80b09fada1a9bb07b1ad62d27074f0b763f041c7d0d98adbcd84e959b'

### 1.1 We know the event

In [6]:
receipt = w3.eth.getTransactionReceipt(txhash)

contract_address = receipt['to'] 
contract_address = Web3.toChecksumAddress(contract_address)
Contract = w3.eth.contract(address=contract_address, abi=ocr_abi)

log = Contract.events.NewTransmission().processReceipt(receipt,errors=DISCARD) #errors=WARN
log[0]['args'] #log data

AttributeDict({'aggregatorRoundId': 601899,
 'answer': 288028518954,
 'transmitter': '0xd0A8Cb58efcee1CAeE48F3c357074862cA8210dc',
 'observations': [287449973102,
  287740000000,
  287874000000,
  287874000000,
  288003015405,
  288028518954,
  288028518954,
  288028518954,
  288028518954,
  288039486674,
  288325243765,
  288330727625,
  288621968577,
  288621968577,
  288673000000,
  288673000000],
 'observers': b'\x0e\x02\x0f\x04\x0c\x01\x03\x08\x00\t\x07\x06\x0b\r\n\x05',
 'rawReportContext': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x93t\x13\xae\xdb\x18\xe5v\x17J\xbf\xf5\x0e\x80\xdcF\x00\x02$\x0b\x01'})

### 1.2 We don't know the event or need all the transaction events

In [19]:
receipt = w3.eth.getTransactionReceipt(txhash)
for l in receipt.logs:
    evt_topic0 = l['topics'][0].hex()
    print(f"LogIndex ={l['logIndex']}, topic={evt_topic0}:")
    try:
        evt_abi = event_topics[event_topics['topic']== evt_topic0]['abi'].values[0] 
        evt = get_event_data(w3.codec, evt_abi, l)
        print(f"Event={evt['event']}")
        display(evt)
    except IndexError:
        print(f"Unrecognised event:")
        print(l)
    finally:
        print('******************************************')    
    

LogIndex =154, topic=0xf6a97944f31ea060dfde0566e4167c1a1082551e64b60ecb14d599a9d023d451:
Event=NewTransmission


AttributeDict({'args': AttributeDict({'aggregatorRoundId': 601899,
  'answer': 288028518954,
  'transmitter': '0xd0A8Cb58efcee1CAeE48F3c357074862cA8210dc',
  'observations': [287449973102,
   287740000000,
   287874000000,
   287874000000,
   288003015405,
   288028518954,
   288028518954,
   288028518954,
   288028518954,
   288039486674,
   288325243765,
   288330727625,
   288621968577,
   288621968577,
   288673000000,
   288673000000],
  'observers': b'\x0e\x02\x0f\x04\x0c\x01\x03\x08\x00\t\x07\x06\x0b\r\n\x05',
  'rawReportContext': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x93t\x13\xae\xdb\x18\xe5v\x17J\xbf\xf5\x0e\x80\xdcF\x00\x02$\x0b\x01'}),
 'event': 'NewTransmission',
 'logIndex': 154,
 'transactionIndex': 18,
 'transactionHash': HexBytes('0x6bf6b3a80b09fada1a9bb07b1ad62d27074f0b763f041c7d0d98adbcd84e959b'),
 'address': '0xE41B5D02E64b165e77f12b72Bf80B56d076000CF',
 'blockHash': HexBytes('0x877dc840bc8b0b34418ff0b54e0e29f872bd1d011f8c684eef68423dc6ae2c82'),
 'blockNumb

******************************************
LogIndex =155, topic=0x0109fc6f55cf40689f02fbaad7af7fe7bbac8a3d2186600afc7d3e10cac60271:
Event=NewRound


AttributeDict({'args': AttributeDict({'roundId': 601899,
  'startedBy': '0x0000000000000000000000000000000000000000',
  'startedAt': 1637555941}),
 'event': 'NewRound',
 'logIndex': 155,
 'transactionIndex': 18,
 'transactionHash': HexBytes('0x6bf6b3a80b09fada1a9bb07b1ad62d27074f0b763f041c7d0d98adbcd84e959b'),
 'address': '0xE41B5D02E64b165e77f12b72Bf80B56d076000CF',
 'blockHash': HexBytes('0x877dc840bc8b0b34418ff0b54e0e29f872bd1d011f8c684eef68423dc6ae2c82'),
 'blockNumber': 21658023})

******************************************
LogIndex =156, topic=0x0559884fd3a460db3073b7fc896cc77986f16e378210ded43186175bf646fc5f:
Event=AnswerUpdated


AttributeDict({'args': AttributeDict({'current': 288028518954,
  'roundId': 601899,
  'updatedAt': 1637555941}),
 'event': 'AnswerUpdated',
 'logIndex': 156,
 'transactionIndex': 18,
 'transactionHash': HexBytes('0x6bf6b3a80b09fada1a9bb07b1ad62d27074f0b763f041c7d0d98adbcd84e959b'),
 'address': '0xE41B5D02E64b165e77f12b72Bf80B56d076000CF',
 'blockHash': HexBytes('0x877dc840bc8b0b34418ff0b54e0e29f872bd1d011f8c684eef68423dc6ae2c82'),
 'blockNumber': 21658023})

******************************************
LogIndex =157, topic=0x4dfe1bbbcf077ddc3e01291eea2d5c70c2b422b415d95645b9adcfd678cb1d63:
Unrecognised event:
AttributeDict({'address': '0x0000000000000000000000000000000000001010', 'topics': [HexBytes('0x4dfe1bbbcf077ddc3e01291eea2d5c70c2b422b415d95645b9adcfd678cb1d63'), HexBytes('0x0000000000000000000000000000000000000000000000000000000000001010'), HexBytes('0x000000000000000000000000d0a8cb58efcee1caee48f3c357074862ca8210dc'), HexBytes('0x000000000000000000000000f0245f6251bef9447a08766b9da2b07b28ad80b0')], 'data': '0x000000000000000000000000000000000000000000000000001ed7189c224c0000000000000000000000000000000000000000000000012f935573d841987759000000000000000000000000000000000000000000000152caa34296f2927ea000000000000000000000000000000000000000000000012f93369cbfa5762b59000000000000000000000000000000000000000000000152cac219af8eb4caa0', 'blockNumber': 21658023, 'transactionHash': HexBytes('0x6bf6b3a80b09fada1a9bb07b1ad62d27074f0b763f041c7d0d98ad

## <font color='blue'> 2. If we need to loop over all the logs for a partuclar contract </font>

In [8]:
contract_address = '0xe41b5d02e64b165e77f12b72bf80b56d076000cf'
contract_address = Web3.toChecksumAddress(contract_address)

In [9]:
#pull all the logs from an address
logs = w3.eth.get_logs({"address": contract_address
                       #,"topics":[NewTransmission_topic] #optional filter
                       ,"fromBlock": w3.eth.get_block('latest')['number'] - 1000 # last 1000 blocks
                       ,"toBlock": 'latest'
                       })

all_events = []
for l in logs:
    try:
        evt_topic0 = l['topics'][0].hex()
        evt_abi = event_topics[event_topics['topic']== evt_topic0]['abi'].values[0] 
        evt = get_event_data(w3.codec, evt_abi, l)
    except MismatchedABI: #if for some reason there are other events 
        pass
    all_events.append(evt)
df = pd.DataFrame(all_events)
df

,address,args,blockHash,blockNumber,event,logIndex,transactionHash,transactionIndex
0,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'gi%\x08i\xcc\xe4\xe9je\xb3d\x8e?\xe7.\xbd\x1...,21738502,NewTransmission,401,b'\xdd\xbca\x98PG\xf6\xc0\xe8g$\xd8\xd7\x89\xd...,42
1,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(roundId, startedBy, startedAt)",b'gi%\x08i\xcc\xe4\xe9je\xb3d\x8e?\xe7.\xbd\x1...,21738502,NewRound,402,b'\xdd\xbca\x98PG\xf6\xc0\xe8g$\xd8\xd7\x89\xd...,42
2,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(current, roundId, updatedAt)",b'gi%\x08i\xcc\xe4\xe9je\xb3d\x8e?\xe7.\xbd\x1...,21738502,AnswerUpdated,403,b'\xdd\xbca\x98PG\xf6\xc0\xe8g$\xd8\xd7\x89\xd...,42
3,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'\xa6sG2\x1e\x18\x1c5\x02/\xf3\xac\x12\xfd\xb...,21738507,NewTransmission,598,b'\x83\x93\x8f\xc4a\x87\xe6lp\t\x9a\xa6\xed\r\...,38
4,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(roundId, startedBy, startedAt)",b'\xa6sG2\x1e\x18\x1c5\x02/\xf3\xac\x12\xfd\xb...,21738507,NewRound,599,b'\x83\x93\x8f\xc4a\x87\xe6lp\t\x9a\xa6\xed\r\...,38
...,...,...,...,...,...,...,...,...
571,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(roundId, startedBy, startedAt)",b'!\xea\x83\x82\xbb\x82\x85x\xd5\x9d\x80)k\xe0...,21739495,NewRound,72,"b'z\xc5\x02U\x07\x93\xff(z`RJ""\xfav\tF\xd8\xd0...",10
572,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(current, roundId, updatedAt)",b'!\xea\x83\x82\xbb\x82\x85x\xd5\x9d\x80)k\xe0...,21739495,AnswerUpdated,73,"b'z\xc5\x02U\x07\x93\xff(z`RJ""\xfav\tF\xd8\xd0...",10
573,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(aggregatorRoundId, answer, transmitter, obser...",b'\xd9\xd9\x9e\x8a\xe6i@\xda\xd0\xe5\x87\x8f\x...,21739497,NewTransmission,616,b'Jf\x85\x95\x0e\x14\x13u\xdc\xd0\xaa\xa0v\xcc...,94
574,0xE41B5D02E64b165e77f12b72Bf80B56d076000CF,"(roundId, startedBy, startedAt)",b'\xd9\xd9\x9e\x8a\xe6i@\xda\xd0\xe5\x87\x8f\x...,21739497,NewRound,617,b'Jf\x85\x95\x0e\x14\x13u\xdc\xd0\xaa\xa0v\xcc...,94


In [10]:
#Counts of parsed events 
df.groupby(['event']).count()['transactionHash']

event
AnswerUpdated      192
NewRound           192
NewTransmission    192
Name: transactionHash, dtype: int64

In [11]:
df.loc[0,'args'] #parsed log data

AttributeDict({'aggregatorRoundId': 617732,
 'answer': 305624201757,
 'transmitter': '0x983D0e1d23D3109D15e833fB800A51ba154DEfD3',
 'observations': [305524000000,
  305548000000,
  305548000000,
  305550000000,
  305550000000,
  305550305288,
  305600408189,
  305624201757,
  305624201757,
  305624201757,
  305634000000,
  305970049484,
  305993193786,
  306048545083,
  306131410533],
 'observers': b'\n\x04\x02\x00\x0f\x0c\t\x0e\x01\x08\r\x0b\x03\x07\x06',
 'rawReportContext': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x93t\x13\xae\xdb\x18\xe5v\x17J\xbf\xf5\x0e\x80\xdcF\x00\x026\x17\x02'})

## <font color='blue'> 3. If we have only event data (no RPC conection required)</font>

### <font size = 3>OCR NewTransmission</font>

In [12]:
#data from database
NewTransmission_data = '000000000000000000000000000000000000000000000000000000582f64b4720000000000000000000000005565b5362ff9f468ba2f144f38b87187c9a010a800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000002c0000000000000000000000066c8885c7645f4815b0bd9981dc827ed00000f900500000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000005810b9f3fd000000000000000000000000000000000000000000000000000000581b70c05b000000000000000000000000000000000000000000000000000000581b87b2c0000000000000000000000000000000000000000000000000000000581d3524990000000000000000000000000000000000000000000000000000005821ce44b3000000000000000000000000000000000000000000000000000000582763a9d2000000000000000000000000000000000000000000000000000000582763a9d2000000000000000000000000000000000000000000000000000000582763a9d2000000000000000000000000000000000000000000000000000000582f64b47200000000000000000000000000000000000000000000000000000058302628c8000000000000000000000000000000000000000000000000000000583765bf13000000000000000000000000000000000000000000000000000000583765bf13000000000000000000000000000000000000000000000000000000583b23b09a000000000000000000000000000000000000000000000000000000583dfe1f170000000000000000000000000000000000000000000000000000005846ea6bc1000000000000000000000000000000000000000000000000000000585dab5cb50000000000000000000000000000000000000000000000000000000000000010070b0f0500040c0a0201080e030d060900000000000000000000000000000000'

In [13]:
event_dict = event_topics[(event_topics['contract_type']=='OCR') & (event_topics['event']=='NewTransmission')]
event_abi = event_dict['abi'].values[0] 
event_topic = event_dict['topic'].values[0] 
event = AttributeDict({'data':NewTransmission_data
                      ,'topics': [HexBytes(event_topic)
                                 ,HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001')
                                 ]
                      ,'address':'' #'0x908edc7e1974ecab1ca7164424bc4cac287d83ad'
                      ,'logIndex':''
                      ,'transactionIndex':''
                      ,'transactionHash':''
                      ,'blockHash':''
                      ,'blockNumber':''
                             
        })
event = get_event_data(default_codec, event_abi, event) # w3.codec->default_codec if we have no rpc connection
event['args']

AttributeDict({'aggregatorRoundId': 1,
 'answer': 378752250994,
 'transmitter': '0x5565b5362FF9f468bA2f144f38b87187C9a010A8',
 'observations': [378237744125,
  378417496155,
  378419000000,
  378447144089,
  378524288179,
  378617965010,
  378617965010,
  378617965010,
  378752250994,
  378764929224,
  378886536979,
  378886536979,
  378949316762,
  378997186327,
  379146890177,
  379528633525],
 'observers': b'\x07\x0b\x0f\x05\x00\x04\x0c\n\x02\x01\x08\x0e\x03\r\x06\t',
 'rawReportContext': b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00f\xc8\x88\\vE\xf4\x81[\x0b\xd9\x98\x1d\xc8'\xed\x00\x00\x0f\x90\x05"})

### <font size = 3>OCR BillingSet</font> 

In [14]:
#data from a database
BillingSet_data = '00000000000000000000000000000000000000000000000000000000000007d0000000000000000000000000000000000000000000000000000000000000006400000000000000000000000000000000000000000000000000000000055484680000000000000000000000000000000000000000000000000000000001ab3f00000000000000000000000000000000000000000000000000000000000a21fe80'

In [15]:
event_dict = event_topics[(event_topics['contract_type']=='OCR') & (event_topics['event']=='BillingSet')]
event_abi = event_dict['abi'].values[0] 
event_topic = event_dict['topic'].values[0] 
event = AttributeDict({'data':BillingSet_data
                                 ,'topics': [HexBytes(event_topic)]
                                 ,'address':'' 
                                 ,'logIndex':''
                                 ,'transactionIndex':''
                                 ,'transactionHash':''
                                 ,'blockHash':''
                                 ,'blockNumber':''
                             
    })
event = get_event_data(default_codec, event_abi, event) # w3.codec->default_codec if we have no rpc connection
event['args']

AttributeDict({'maximumGasPrice': 2000,
 'reasonableGasPrice': 100,
 'microLinkPerEth': 89425000,
 'linkGweiPerObservation': 28000000,
 'linkGweiPerTransmission': 170000000})

### <font size = 3>Keepers UpkeepPerformed</font> 

In [16]:
#data from a database
UpkeepPerformed_data ='0x00000000000000000000000000000000000000000000000000010559c9cf7de300000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000000'


In [17]:
event_dict = event_topics[(event_topics['contract_type']=='Keepers') & (event_topics['event']=='UpkeepPerformed')]
event_abi = event_dict['abi'].values[0]  
event_topic = event_dict['topic'].values[0]  
event = AttributeDict({'data':UpkeepPerformed_data
                                ,'topics': [HexBytes(event_topic),
                                            HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001'),
                                            HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001'),
                                            HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001'),
                                           ]
                                ,'address':'' 
                                ,'logIndex':''
                                ,'transactionIndex':''
                                ,'transactionHash':''
                                ,'blockHash':''
                                ,'blockNumber':''
                             
    })
event = get_event_data(default_codec, event_abi, event) # w3.codec->default_codec if we have no rpc connection
event['args']

AttributeDict({'id': 1,
 'success': True,
 'from': '0x0000000000000000000000000000000000000001',
 'payment': 287358172757475,
 'performData': b''})